In [39]:
from struct import *

## Encoder

In [40]:
def vb_encode_number(n):
    # 全ての数字を４バイトや８バイトに固定するのでは無く、必要最小限のバイトにする
    # ただし、各バイトの最大桁のビットに関しては、フラグとして使うため、1byte = 7bitになる
    # bytesの各要素は、bit数を10進数で表す(合ってるよな?)
    bytes = list()
    while True:
        bytes.insert(0, n%128)
        n = n // 128
        if n == 0:
            break
    bytes[-1] += 128
    return bytes

In [41]:
# 12 = 10001100
assert vb_encode_number(12) == [140]
# 130 = 00000001 10000010
assert vb_encode_number(130) == [1, 130]

In [42]:
def vb_encode(list_n):
    byte_stream = []
    
    # len(lisn_n) >= 1 という前提で実装する
    byte_stream.extend(vb_encode_number(list_n[0]))
    
    # list_nは短調増加するという前提で実装する
    previous_n = list_n[0]
    for n in list_n[1:]:
        byte_stream.extend(vb_encode_number(n - previous_n))
        previous_n = n
    return byte_stream

In [43]:
# 12, 142 => 12, 130 => [12], [1, 130]
assert vb_encode([12, 142]) == [140, 1, 130]
# 12, 142, 152 => 12,130,10 => [140, 1, 130, 138]
assert vb_encode([12,142,152]) == [140, 1, 130, 138]

# これらをpackする

In [44]:
def vb_encode_file(filename):
    all_bytes = bytes()
    with open("./{}".format(filename)) as f:
        lines = f.read().splitlines()
    for line in lines:
        bytes_tag = line.split(' ')[0].encode()
        print(bytes_tag)
        # ファイルの各行のタグ(文字列)をバイト配列にした時、それが何バイトに相当するか
        # iなので、4byte分使う(つまり、tagの長さは最大2の32乗バイトで表せる文字列までが実質許容される)
        all_bytes += pack('i', len(bytes_tag))
        
        # 数列を変換したバイト配列
        byte_stream = vb_encode(list(map(lambda x: int(x.strip()), line.split(' ', 1)[1].split(','))))
        
        # 4byte使用して、バイト配列が幾つの要素からなるかを表す(それぞれの要素は1byteに変換される)
        # iなので、4byte分使う(つまり実質、、、)
        all_bytes += pack('i', len(byte_stream))
        
        # tagをバイト配列にする
        all_bytes += bytes_tag
        
        # バイト配列
        all_bytes += pack("{}B".format(len(byte_stream)), *byte_stream)

#         all_bytes += byte_stream
#         list_byte_stream.extend(pack("{}L".format(len(byte_stream)+2), len(tag), len(byte_stream)))
#         list_byte_stream.extend(lenbyte_stream)
    with open('./encoded', mode='wb') as f:
        # ファイルサイズと要素数はそれぞれ、unsigned long(256の4乗-1)以下という前提で実装する
        f.write(all_bytes)
#         f.write(pack('l' , 1))
#     return pack("{}L".format(len(byte_stream)+2), len(tag), len(byte_stream), *byte_stream)


vb_encode_file('./original.csv')

b'aaa'
b'bbb'


In [45]:
pack('i', 3)

b'\x03\x00\x00\x00'

In [46]:
unpack('l', b'\x03\x00\x00\x00\x00\x00\x00\x00')

(3,)

`aaa 1, 130, 142` だと、`03 00 00 00 04 00 00 00 61 61 61 8C 01 82 8A`

- 最初の4byte`03 00 00 00`がタグが何バイト分取っているかを表すの要素数(3)
- 次の4byte`04 00 00 00`がバイト配列の要素数を表す(4)
- 次の3byte`61 61 61`が'aaa'をバイト配列にしたもの
- 残りの4byte`8C 01 82 8A`が数列(1, 129, 12)のバイト配列

## Decoder

In [68]:
def vb_decode_file(filename):
    with open("./{}".format(filename), mode='rb') as f:
        len_tag = unpack('i', f.read(4))[0]
        len_bytes = unpack('i', f.read(4))[0]
        print(len_tag, len_bytes)
        
        # 普通に
        print(f.read(1))
        print(unpack('c'*len_tag, f.read(len_tag)))
        for _ in range(len_bytes):
            print('a')
        print(f.read(len_bytes))
#         tag = unpack('c', f.read(len_tag))
        
vb_decode_file('encoded')

3 4
b'a'
(b'a', b'a', b'\x8c')
a
a
a
a
b'\x01\x82\x8a\x03'


In [97]:
pack('f', 12.2)

b'33CA'

In [90]:
print(hex(b'abc'))

TypeError: 'bytes' object cannot be interpreted as an integer

In [103]:
type(b'aa\あ')

SyntaxError: bytes can only contain ASCII literal characters. (<ipython-input-103-8d084f527c5d>, line 1)

In [101]:
type(bytearray.fromhex('2Ef0 F1f2  '))

bytearray

In [104]:
bytes(10)

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [111]:
bytes(range(256))

b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x7f\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x90\x91\x92\x93\x94\x95\x96\x97\x98\x99\x9a\x9b\x9c\x9d\x9e\x9f\xa0\xa1\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xab\xac\xad\xae\xaf\xb0\xb1\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xbb\xbc\xbd\xbe\xbf\xc0\xc1\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xcb\xcc\xcd\xce\xcf\xd0\xd1\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xdb\xdc\xdd\xde\xdf\xe0\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xeb\xec\xed\xee\xef\xf0\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xfb\xfc\xfd\xfe\xff'

In [118]:
for i in range(256):
    print(bytes(range(256))[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
